In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_lattice as tfl
import numpy as np
import pandas as pd

np.random.seed(1)

In [2]:
# DATA
df = pd.read_csv('C:/Users/tln229/Downloads/Python/1. Building/data/HVAC_B90_102_exp_10m_20210424.csv')

In [3]:
for n_train in ([32, 64, 128]):
    for y_max in ([2, 5, 10, 20]):
        for grid_size in ([3, 10, 20, 50]):
            for lr in ([0.001, 0.01, 0.1, 0.2]):
                
                # DATA
                # trainning set
                Tca_k1   = np.array(df['ahu_supply_temp']).reshape(-1,1)[1: n_train+1]
                Tsa_k    = np.array(df['supply_discharge_temp']).reshape(-1,1)[0: n_train]
                Tsa_k1   = np.array(df['supply_discharge_temp']).reshape(-1,1)[1: n_train+1]
                valve_k2 = np.array(df['htg_valve_position']).reshape(-1,1)[2: n_train+2]

                X = np.concatenate((Tca_k1, Tsa_k, Tsa_k1, valve_k2), axis=1)
                Y = np.array(df['supply_discharge_temp']).reshape(-1,1)[2: n_train+2]

                # test set
                Tca_k1   = np.array(df['ahu_supply_temp']).reshape(-1,1)[n_train+1: 1600+1]
                Tsa_k    = np.array(df['supply_discharge_temp']).reshape(-1,1)[n_train: 1600]
                Tsa_k1   = np.array(df['supply_discharge_temp']).reshape(-1,1)[n_train+1: 1600+1]
                valve_k2 = np.array(df['htg_valve_position']).reshape(-1,1)[n_train+2: 1600+2]

                X_t = np.concatenate((Tca_k1, Tsa_k, Tsa_k1, valve_k2), axis=1)
                Y_t = np.array(df['supply_discharge_temp']).reshape(-1,1)[n_train+2: 1600+2]

                # MODEL
                # combined calibrator
                combined_calibrators = tfl.layers.ParallelCombination()

                calibrator_x1 = tfl.layers.PWLCalibration(input_keypoints=np.linspace(53, 57, grid_size), output_min=0, output_max=y_max, monotonicity='increasing')
                calibrator_x2 = tfl.layers.PWLCalibration(input_keypoints=np.linspace(55, 96, grid_size), output_min=0, output_max=y_max)
                calibrator_x3 = tfl.layers.PWLCalibration(input_keypoints=np.linspace(55, 96, grid_size), output_min=0, output_max=y_max)
                calibrator_x4 = tfl.layers.PWLCalibration(input_keypoints=np.linspace(0, 100, grid_size), output_min=0, output_max=y_max, monotonicity='increasing')

                combined_calibrators.append(calibrator_x1)
                combined_calibrators.append(calibrator_x2)
                combined_calibrators.append(calibrator_x3)
                combined_calibrators.append(calibrator_x4)

                # Lattice
                lattice = tfl.layers.Lattice(
                    lattice_sizes = [y_max+1, y_max+1, y_max+1, y_max+1],
                    monotonicities = ['increasing', 'increasing', 'increasing', 'increasing'],
                    output_min = 55,
                    output_max = 96)
                
                # model
                inps  = tf.keras.Input(shape=(4,))
                calib = combined_calibrators(inps)
                outs  = lattice(calib)

                model = tf.keras.Model(inputs=inps, outputs=outs)
                model.compile(
                    loss = tf.keras.losses.mean_squared_error,
                    optimizer = tf.keras.optimizers.Adam(learning_rate=lr),
                    metrics = [tfa.metrics.RSquare()])
                
                # TRAIN
                callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=500)

                hist = model.fit(
                    x = X,
                    y = Y,
                    batch_size = Y.shape[0],
                    epochs = 200000,
                    shuffle = False,
                    validation_data=(X_t, Y_t),
                    callbacks=[callback],
                    verbose = 0)
                
                a = np.argmax(hist.history['val_r_square'])
                print('n:%3d  \t  y_max:%2d  \t  grid_size:%3d  \t  lr:%6.4f  \t  r2:%7.4f' % (n_train, y_max, grid_size, lr, hist.history['val_r_square'][a]))

n: 32  	  y_max: 2  	  grid_size:  3  	  lr:0.0010  	  r2: 0.8916
n: 32  	  y_max: 2  	  grid_size:  3  	  lr:0.0100  	  r2: 0.8976
n: 32  	  y_max: 2  	  grid_size:  3  	  lr:0.1000  	  r2: 0.9001
n: 32  	  y_max: 2  	  grid_size:  3  	  lr:0.2000  	  r2: 0.9016
n: 32  	  y_max: 2  	  grid_size: 10  	  lr:0.0010  	  r2: 0.9211
n: 32  	  y_max: 2  	  grid_size: 10  	  lr:0.0100  	  r2: 0.9217
n: 32  	  y_max: 2  	  grid_size: 10  	  lr:0.1000  	  r2: 0.8881
n: 32  	  y_max: 2  	  grid_size: 10  	  lr:0.2000  	  r2: 0.9062
n: 32  	  y_max: 2  	  grid_size: 20  	  lr:0.0010  	  r2: 0.9284
n: 32  	  y_max: 2  	  grid_size: 20  	  lr:0.0100  	  r2: 0.9298
n: 32  	  y_max: 2  	  grid_size: 20  	  lr:0.1000  	  r2: 0.8776
n: 32  	  y_max: 2  	  grid_size: 20  	  lr:0.2000  	  r2: 0.8604
n: 32  	  y_max: 2  	  grid_size: 50  	  lr:0.0010  	  r2: 0.9313
n: 32  	  y_max: 2  	  grid_size: 50  	  lr:0.0100  	  r2: 0.9272
n: 32  	  y_max: 2  	  grid_size: 50  	  lr:0.1000  	  r2: 0.7488
n: 32  	  